# Homework 2

# Set up

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [ ]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
# DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

In [ ]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    # print("=" * 80)
    # print(f"📄 {pdf_name}")
    # print("=" * 80)
    # print(result.text_content)
    # print("\n\n")


# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [ ]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
# for tool in mcp_tools:
#     print(tool.name)
#     print(tool.description)
#     print(tool.args)
#     print("\n\n------------------------------------------------------\n\n")

## A simple agent using tools from the MCP server


In [ ]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY, # Ensure this key is set in Colab secrets
    temperature=0,
    vertexai=True
)
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def get_tool_by_name(tool_name: str):
    for t in tools:
        if t.name == tool_name:
            return t
search_linkedin_people = get_tool_by_name("search_linkedin_people")
get_linkedin_profile = get_tool_by_name("get_linkedin_profile")
search_facebook_users = get_tool_by_name("search_facebook_users")
get_facebook_profile = get_tool_by_name("get_facebook_profile")

In [ ]:
# This block provides you some tests to get faminilar with our MCP server

# # Test 1: Search Facebook users (exact match)
# await tools[0].ainvoke({'q': "Minh Pham", 'limit': 5})

# # Test 2: Search Facebook users (fuzzy match with typo)
# await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# # Test 3: Get Facebook profile
# await tools[1].ainvoke({'user_id': 180})

# # Test 4: Get Facebook mutual friends
# await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
# await tools[3].ainvoke({'q': "Minh Pham", 'location': "Beijing", 'limit': 5, 'fuzzy': True})

# # Test 7: Get LinkedIn profile
# await tools[4].ainvoke({'person_id': 95})

# Test 8: Get LinkedIn interactions
# await tools[5].ainvoke({'person_id': 456})

In [ ]:
debug_enabled = True

In [ ]:
from dataclasses import fields
import json
from typing import List, Optional
from langchain_core.messages import BaseMessage
import re


def safe_invoke(model, cls, input: str, config: dict = None, stop: list = None, retries: int = 3) -> str:
    for attempt in range(1, retries + 1):
        try:
            response = model.invoke(input, config=config, stop=stop)
            content = response.content.strip()
            content = re.sub(r"^```(?:json)?\s*|\s*```$", "", content.strip())
            data = json.loads(content)
            field_names = {f.name for f in fields(cls)}
            filtered_data = {k: v for k, v in data.items() if k in field_names}
            missing = field_names - filtered_data.keys()
            if missing:
                raise ValueError(f"Missing fields: {missing}")
            if debug_enabled:
                print(f"[safe_invoke] response: {json.dumps(filtered_data, indent=2, ensure_ascii=False)}")
            return filtered_data
        except json.JSONDecodeError as e:
            print(f"[safe_invoke:{attempt}] JSON decode error: {str(e)}. Response content: {response.content}")
        except Exception as e:
            print(f"[safe_invoke:{attempt}] Error during LLM invocation: {str(e)}")
    return {f.name: None for f in fields(cls)}

def describe_schema(cls):
    lines = ["Your response **must** be a JSON object with the following format:"]
    for f in fields(cls):
        if f.name.startswith("template_"):
            continue
        desc = f.metadata.get("description", "")
        lines.append(f'- {f.name} ({f.type.__name__}): {desc}')
    if hasattr(cls, "output_format"):
        lines.append(f"Here is an example of the expected output format:{cls.output_format}")
    return "\n".join(lines)

Define state

In [ ]:
from typing import TypedDict, List, Dict, Any, Optional, Annotated, operator

class CVState(TypedDict):
    cv_file: str
    cv_text: str
    cv_json: dict
    report: Optional[str]

    linkedin_id: int | None
    facebook_id: int | None
    linkedin_profile: Dict[str, Any] | None
    facebook_profile: Dict[str, Any] | None
    
    discrepancies: Annotated[List[Dict[str, Any]], operator.add]

    trust_score: Optional[float]

Define response format

In [ ]:
from dataclasses import dataclass, field

@dataclass
class OrganizeOutput:
    # thought: str = field(metadata={"description": "One sentence thought process."})
    personal_info: dict = field(metadata={"description": "Personal information including name, professional summary, and locations."})
    education: List[dict] = field(metadata={"description": "List of educational background entries, each containing institution, degree, and graduation year."})
    experience: List[dict] = field(metadata={"description": "List of work experience entries, each containing company, job title, period, and responsibilities."})
    skills: List[str] = field(metadata={"description": "List of skills."})

    output_format = """
{
  "personal_info": {
    "name": <string>,
    "professional_summary": <string>,
    "locations": ["<string>", ...]
  },
  "education": [
    {
      "institution": <string>,
      "degree": <string>,
      "graduation_year": <string>
    },
    ...
  ],
  "experience": [
    {
      "company": "<string>",
      "job_title": "<string>",
      "period": "<string>",
      "responsibilities": ["<string>", ...]
    },
    ...
  ],
  "skills": ["<string>", "..."],
}
"""

@dataclass
class SearchOutput:
    thought: str = field(metadata={"description": "One sentence thought process."})
    id: str = field(metadata={"description": "Candidate ID found from LinkedIn or Facebook."})

    output_format = """
{
  "thought": <string>,
  "id": <string>
}
"""

@dataclass
class InvestigateOutput:
    discrepancies: list[dict] = field(metadata={"description": "Discrepancies found between CV and LinkedIn profile."})

    output_format = """
{
  "discrepancies": [
    {
      "severity": <string>,
      "reason": <string>
    },
    ...
  ]
}
"""

In [ ]:
async def search_candidates(cv_locations: list, cv_name: str, cv_industry: str) -> list:
    all_candidates = {}
    locations_to_search = cv_locations if cv_locations else [None]
    for loc in locations_to_search:
        try:
            results = await search_linkedin_people.ainvoke({
                "q": cv_name,
                "location": loc,
                "industry": cv_industry,
                "limit": 10,
                "fuzzy": True
            })
            
            for cand in results:
                all_candidates[cand['id']] = cand
        except Exception as e:
            print(f"Error when search for '{cv_name}','{loc}': {e}")
            
    return list(all_candidates.values())

Define graph nodes

In [ ]:
def organize_node(state: CVState):
    print("[organize_node] Organizing CV information into structured format...")
    prompt = f"""
You are a helpful assistant that extracts structured information from CV text.

[Task]
organize the following information from the CV text into structured JSON format

[Extracted CV Text]
{state["cv_text"]}

[Output Format]
{describe_schema(OrganizeOutput)}

[Warning]
- For locations, if only exist country, just return country. If city and country are available, **Only** return city. DO NOT indicate "(Hometown)" or "(Current Location)". Just return location name.
- You cannot leave any field empty.
"""
    response = safe_invoke(llm, OrganizeOutput, prompt)
    return {
        "cv_json": response
    }

async def search_node(state: CVState):
    print("[search_node] Searching for LinkedIn and Facebook profiles...")
    cv = state["cv_json"]
    candidates = await search_candidates(
        cv_locations=cv['personal_info'].get('locations', []),
        cv_name=cv['personal_info']['name'],
        cv_industry=cv['personal_info'].get('professional_summary', '').split()[0]
    )
    print(f"[search_node] candidates for {cv['personal_info']['name']}: {candidates}")
    if not candidates:
        print(f"[search_node] No candidates found in search_node for {cv['personal_info']['name']}.")
        return {"linkedin_id": None, "linkedin_profile": None}
    prompt = f"""
You are a professional background check expert. Please compare the job seeker's CV information with the following list of candidates found on LinkedIn.

[CV Information]
Name: {cv['personal_info']['name']}
Summary: {cv['personal_info'].get('professional_summary', 'None')}
Locations: {', '.join(cv['personal_info'].get('locations', []))}

[LinkedIn Search Results]
{json.dumps(candidates, ensure_ascii=False, indent=2)}
Please analyze which candidate is **MOST LIKELY** the job seeker.

[Output Format]
{describe_schema(SearchOutput)}
"""
    print(f"[search_node] Prompt for candidate search:\n{prompt}")
    response = safe_invoke(llm, SearchOutput, prompt)
    target_id = response['id']
    candidate_profile = await get_linkedin_profile.ainvoke({"person_id": target_id})
    return {"linkedin_id": target_id, "linkedin_profile": candidate_profile}

async def investigate_node(state: CVState):
    print("[investigate_node] Investigating discrepancies between CV and LinkedIn profile...")
    linkedin_profile = state["linkedin_profile"]
    if linkedin_profile is None:
        print("[investigate_node] No LinkedIn profile to investigate.")
        return {"discrepancies": [{"severity": "high", "reason": "No LinkedIn profile found for investigation.", "penalty": 0.6}], "trust_score": 0.4}
    cv = state["cv_json"]
    prompt = f"""
You are a professional background check expert. Please compare the job seeker's CV information with the LinkedIn profile information.
[CV Information]
{cv}
[LinkedIn Profile Information]
{linkedin_profile}
[scoring criteria]
Each cv credibility is scored between [0, 1]. 1.0 indicates perfect match and no discrepancies. 0.0 indicates major discrepancies. You need to identify all mismatches and deduct points according to the following rules:

1. Core Resume Falsification (High, -0.3): The company listed on the CV does not exist on LinkedIn.
2. Education Falsification (High, -0.3): The school or degree is seriously inconsistent.
3. Exaggerated Job Title (Medium, -0.15): The CV states Senior/Director, but LinkedIn lists it as Junior/Parallel.
4. Exaggerated Work Hours (Medium, -0.15): The work hours listed on the CV do not match those on LinkedIn.
5. Overstated Skills (Low, -0.05): The core skills listed on the CV do not match those on LinkedIn.
[Output Format]
{describe_schema(InvestigateOutput)}
"""
    print(f"[investigate_node] Prompt for investigation:\n{prompt}")
    response = safe_invoke(llm, InvestigateOutput, prompt)
    disparencies = response["discrepancies"]
    trust_score = 1.0
    for d in disparencies:
        severity = d.get("severity", "low").lower()
        if severity == "high":
            trust_score -= 0.3
        elif severity == "medium":
            trust_score -= 0.15
        elif severity == "low":
            trust_score -= 0.05
    trust_score = max(0.0, min(1.0, trust_score))
    return {
        "discrepancies": response["discrepancies"],
        "trust_score": trust_score
    }

In [ ]:
from langgraph.graph import StateGraph, END

agent_builder = StateGraph(CVState)
agent_builder.add_node("organize", organize_node)
agent_builder.add_node("search", search_node)
agent_builder.add_node("investigate", investigate_node)
agent_builder.add_edge("organize", "search")
agent_builder.add_edge("search", "investigate")
agent_builder.add_edge("investigate", END)

agent_builder.set_entry_point("organize")
agent_graph = agent_builder.compile()

In [ ]:
inputs = [{"cv_file": cv["file"], "cv_text": cv["text"]} for cv in all_cvs]
results = await agent_graph.abatch(inputs,)

In [ ]:
import re
def extract_cv_index(filename: str) -> int:
    match = re.search(r'CV_(\d+)\.pdf', filename)
    return int(match.group(1)) if match else -1

results_sorted = sorted(
    results,
    key=lambda r: extract_cv_index(r["cv_file"])
)

scores = [r["trust_score"] for r in results_sorted]
scores = [r["trust_score"] for r in results]

# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [ ]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [ ]:
# scores = ... # Your code should generate this list [0.2, 0.3, 0.4, 0.5, 0.6]
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)
